### В этом файле содержится код для генерации матриц, используемых для проверки и сравнения методов.

In [1]:
import numpy as np
from math import sqrt

Вспомогательная функция для красивого вывода матриц.

In [2]:
def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")

Зададим размер матриц.

In [3]:
N = 10

Добавим ряд проверок для будущих матриц.

In [4]:
def is_symm(x):
    for i in range(len(x)):
        for j in range(len(x)):
            if (not np.isclose(x[i][j], x[j][i])):
                return False
    return True

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

def is_diag_dom(x):    
    D = np.diag(np.abs(x))
    S = np.sum(np.abs(x), axis=1) - D
    return np.all(D > S)

def is_norm(x, a, b):
    return np.all(np.logical_and(x >= a, x <= b))

%%latex

Научимся генерировать случайную симметричную положительную матрицу с числами из диапазона 0-1. Для этого сгенерируем случайную матрицу с числами из диапазона 0-1, "отзеркалим", и поделим ее на $\sqrt{n}$.

In [5]:
def gen_random(n):
    a = np.random.rand(n, n)
    m = np.tril(a) + np.tril(a, -1).T
    m = np.true_divide(m, sqrt(n))
    return np.dot(m, m.transpose())

Научимся генерировать хорошо обусловленную матрицу, в которой для каждого элемента диагонали выполняется строгое равенство.

In [6]:
def gen_well_conditioned(n):
    a = gen_random(n)
    D = 0.5
    for i in range(n):
        a[i][i] = sum(a[i]) - a[i][i] + D
    return a

Научимся генерировать плохо обусловленную матрицу. Тривиально.

In [7]:
def gen_poorly_conditioned(n):
    return np.array([[1 / (1 + i + j) for i in range(n)] for j in range(n)])

Сгенерируем столбец свободных коэффициентов, общий для всех матриц.

In [8]:
b = np.random.rand(N)

**Сгенерируем все 3 матрицы, для каждой выполним проверку на соответствие заданным условиям и сохраним их в файл. Эти матрицы будут в дальнейшем использоваться для проверки различных методов и их сравнения.**

Хорошо обусловленная матрица:

In [9]:
well = gen_well_conditioned(N)
assert(is_symm(well))
assert(is_pos_def(well))
assert(is_diag_dom(well))
matprint(well)

 2.82622  0.155787  0.316136  0.278254   0.27169  0.220576  0.221638  0.328687  0.235647  0.297809  
0.155787   2.13939  0.200077   0.20067  0.157949  0.156371  0.144816  0.211407  0.216417  0.195898  
0.316136  0.200077   3.35898  0.354802  0.329449  0.278091   0.32436  0.365637  0.323395  0.367031  
0.278254   0.20067  0.354802   3.08394  0.267778  0.243274  0.260838  0.307294   0.28327  0.387764  
 0.27169  0.157949  0.329449  0.267778   2.83571  0.278382  0.241249  0.310632  0.207521  0.271064  
0.220576  0.156371  0.278091  0.243274  0.278382   2.55615  0.228674  0.250451  0.177442  0.222893  
0.221638  0.144816   0.32436  0.260838  0.241249  0.228674   2.63271  0.240184  0.215661  0.255285  
0.328687  0.211407  0.365637  0.307294  0.310632  0.250451  0.240184   3.14272   0.30147  0.326954  
0.235647  0.216417  0.323395   0.28327  0.207521  0.177442  0.215661   0.30147    2.7568  0.295972  
0.297809  0.195898  0.367031  0.387764  0.271064  0.222893  0.255285  0.326954  0.295972   

Рандомная матрица:

In [10]:
random = gen_random(N)
assert(is_symm(random))
assert(is_pos_def(random))
assert(is_norm(random, 0, 1))
matprint(random)

0.297726  0.176547  0.287715  0.273211   0.23333  0.247274   0.129323  0.231289   0.134482  0.189552  
0.176547  0.204856  0.225844  0.194821  0.188989  0.200112   0.103468  0.143333   0.141969  0.158657  
0.287715  0.225844  0.415294   0.33006  0.349434  0.328657   0.151298   0.27527   0.293999   0.27677  
0.273211  0.194821   0.33006  0.420604  0.330291  0.317661   0.168649  0.256893   0.261133  0.233656  
 0.23333  0.188989  0.349434  0.330291  0.454602  0.280871   0.196723  0.302325   0.302345  0.288725  
0.247274  0.200112  0.328657  0.317661  0.280871  0.307591   0.150271  0.198974   0.227854   0.22271  
0.129323  0.103468  0.151298  0.168649  0.196723  0.150271    0.19079  0.162144  0.0815295  0.188212  
0.231289  0.143333   0.27527  0.256893  0.302325  0.198974   0.162144  0.303542   0.190971  0.242407  
0.134482  0.141969  0.293999  0.261133  0.302345  0.227854  0.0815295  0.190971   0.314891  0.202703  
0.189552  0.158657   0.27677  0.233656  0.288725   0.22271   0.188212  0.

Очень плохо обусловленная матрица:

In [11]:
poor = gen_poorly_conditioned(N)
assert(is_symm(poor))
assert(is_pos_def(poor))
matprint(poor)

       1        0.5   0.333333       0.25        0.2   0.166667   0.142857      0.125   0.111111        0.1  
     0.5   0.333333       0.25        0.2   0.166667   0.142857      0.125   0.111111        0.1  0.0909091  
0.333333       0.25        0.2   0.166667   0.142857      0.125   0.111111        0.1  0.0909091  0.0833333  
    0.25        0.2   0.166667   0.142857      0.125   0.111111        0.1  0.0909091  0.0833333  0.0769231  
     0.2   0.166667   0.142857      0.125   0.111111        0.1  0.0909091  0.0833333  0.0769231  0.0714286  
0.166667   0.142857      0.125   0.111111        0.1  0.0909091  0.0833333  0.0769231  0.0714286  0.0666667  
0.142857      0.125   0.111111        0.1  0.0909091  0.0833333  0.0769231  0.0714286  0.0666667     0.0625  
   0.125   0.111111        0.1  0.0909091  0.0833333  0.0769231  0.0714286  0.0666667     0.0625  0.0588235  
0.111111        0.1  0.0909091  0.0833333  0.0769231  0.0714286  0.0666667     0.0625  0.0588235  0.0555556  
     0.1  

In [12]:
delimiter = ','
np.savetxt('well.txt', well, delimiter=delimiter)
np.savetxt('random.txt', random, delimiter=delimiter)
np.savetxt('poor.txt', poor, delimiter=delimiter)
np.savetxt('right.txt', b, delimiter=delimiter)